# <center> Modèle d'apprentissage Train/Val </center>

In [ ]:
from bigml.api import BigML
from statistics import mean
import matplotlib.pyplot as plt

In [ ]:
api = BigML(project= 'project/5d94a407eba31d45c8000088')

In [ ]:
source = api.create_source('../handson-ml2/trainfull.csv')

In [ ]:
origin_dataset = api.create_dataset(source)

In [ ]:
train_dataset = api.create_dataset (origin_dataset, {"name": "Train80", "sample_rate":0.8, "seed": "myseed"})

In [ ]:
val_dataset = api.create_dataset (origin_dataset, {"name": "Val", "sample_rate":0.8, "seed": "myseed", "out_of_bag": True})

In [ ]:
# Learning Curves Split en 10%
x_ensemble = []
y_ensemble = []
x_deepnet = []
y_deepnet = []
for i in range(1,11): 
    # On crée le nouveau dataset de train à partir du dataset de train80 en itérant les 10 %
    dataset_train_split = api.create_dataset(train_dataset, {"name": "Dataset Train Split" + str(i/10), "sample_rate": i/10, "seed": "my seed"})    
    # On crée le modèle (ensemble) à partir du dataset spliter
    ensemble = api.create_ensemble(dataset_train_split, {"objective_field" : "SeriousDlqin2yrs", "name": "Ensemble " + str(i/10)})
    api.ok(ensemble)
    # On évalue le nouveau modèle (ensemble) qui va nous donner l'AUC
    evaluation = api.create_evaluation(ensemble, val_dataset )
    api.ok(evaluation)
    # On fait la même chose pour deepnet
    deepnet = api.create_deepnet(dataset_train_split, {"objective_field" : "SeriousDlqin2yrs", "name": "Deepnet " + str(i/10)})
    api.ok(deepnet)
    evaluation_deepnet = api.create_evaluation(deepnet, val_dataset )
    api.ok(evaluation_deepnet)
    
    x_ensemble.append(i*10)
    y_ensemble.append(evaluation['object']['result']['model']['average_area_under_roc_curve'])
    x_deepnet.append(i*10)
    y_deepnet.append(evaluation_deepnet['object']['result']['model']['average_area_under_roc_curve'])
    i += 1

print(x_ensemble)
print(y_ensemble)
print(x_deepnet)
print(y_deepnet)

In [ ]:
g1 = plt.plot(x_ensemble, y_ensemble, color='darkblue', label ='ensemble')
plt.ylabel('AUC')
plt.xlabel('Taille du dataset train en %')
g2 = plt.plot(x_deepnet, y_deepnet, color='red', label ='deepnet')
plt.title('Learning Curve')
plt.legend()

*<center>Lancement d'un Modèle ou d'un Deepnet ou autre...</center>*

In [ ]:
ensemble = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})
api.ok(ensemble)

In [ ]:
deepnet = api.create_deepnet(train_dataset, {"objective_field":"SeriousDlqin2yrs"})
api.ok(deepnet)

In [ ]:
optiml = api.create_optiml(train_dataset, {"objective_field":"SeriousDlqin2yrs", "metric": "accuracy"})
api.ok(optiml)
api.status(optiml)

*<center> Evaluation du modèle </center>*

In [ ]:
evaluation = api.create_evaluation(ensemble, val_dataset)
api.status(evaluation)
api.ok(evaluation)
evaluation = api.get_evaluation(evaluation)
api.pprint(evaluation['object']['result'])

In [ ]:
evaluation2 = api.create_evaluation(deepnet, val_dataset)
api.status(evaluation2)
api.ok(evaluation2)
evaluation2 = api.get_evaluation(evaluation2)
api.pprint(evaluation2['object']['result'])

*<center>Lancer la prediction à partir de notre modèle en demandant les probabilités</center>*

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, val_dataset, {"all_fields": True, "header": True, "probabilities": True})
api.ok(batch_prediction)

In [ ]:
batch_prediction2 = api.create_batch_prediction(deepnet, val_dataset, {"all_fields": True, "header": True, "probabilities": True})
api.ok(batch_prediction2)

*<center>Récupérer la batch prediction au format csv</center>*

In [ ]:
api.download_batch_prediction(batch_prediction, filename='../handson-ml2/my_predictionsValEnsemble.csv')

In [ ]:
api.download_batch_prediction(batch_prediction2, filename='../handson-ml2/my_predictionsValDeepnet.csv')